In [1]:
import re, os, string, warnings, json
warnings.filterwarnings("ignore")
import math
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt', quiet=True)
from collections import Counter
from rapidfuzz import fuzz
from word2number import w2n
import dateparser
from dateutil import parser as duparser

In [2]:
df = pd.read_csv("synthetic_dataset.csv", encoding_errors="replace")

In [3]:
# This is being done so as to confirm to a same schema for data throughout the project
# Heuristic rename if your headers differ slightly
rename_map = {
    "reference": "gold_reference",
    "gold": "gold_reference",
    "answer": "ai_response",
    "response": "ai_response",
}
for k,v in rename_map.items():
    if k in df.columns and v not in df.columns:
        df.rename(columns={k:v}, inplace=True)

# Ensure exactly the 4 we use exist
for c in ["question","context","gold_reference","ai_response"]:
    if c not in df.columns: df[c] = ""

def _clean(s): 
    return (str(s) if pd.notna(s) else "").strip()

UNANSWERABLE_MARKERS = {
    "unanswerable","not answerable","cannot be determined","insufficient context",
    "not available in passage","unknown","na","n/a"
}

def derive_flags(row):
    gold = _clean(row["gold_reference"]).lower()
    ctx  = _clean(row["context"])
    is_unanswerable = gold in UNANSWERABLE_MARKERS
    context_missing = (ctx == "")
    no_gold = (gold == "")  # empty gold means judge vs context only
    return pd.Series({
        "is_unanswerable": is_unanswerable,
        "context_missing": context_missing,
        "no_gold": no_gold
    })

flags = df.apply(derive_flags, axis=1)
df = pd.concat([df, flags], axis=1)
df.head(3)

question  \
0  What did the Court of Justice not acknowledge?   
1  What did the Court of Justice not acknowledge?   
2  What did the Court of Justice not acknowledge?   

                                             context gold_reference  \
0  Fourth, national courts have a duty to interpr...            NaN   
1  Fourth, national courts have a duty to interpr...            NaN   
2  Fourth, national courts have a duty to interpr...            NaN   

                                         ai_response  is_unanswerable  \
0                    The passage doesn’t state this.            False   
1  It states that what did the court of justice w...            False   
2  The decision was influenced by the Paris Conve...            False   

   context_missing  no_gold  
0            False     True  
1            False     True  
2            False     True

In [ ]:
# Accuracy Scoring
# Accuracy measures how close the AI response is to the gold reference answer.
# It is only computed when a gold_reference is available (if no_gold → score = 0).

# Our accuracy pipeline uses multiple complementary checks:
# 1. Exact Match (EM): Whether the response text exactly matches the gold reference.
# 2. Token-level F1: Measures overlap of words between response and reference.
# 3. ROUGE-L: Longest common subsequence similarity, capturing phrasing differences.
# 4. Semantic Similarity to capture paraphrasing.
# By combining exact, lexical, and semantic overlap, the score accounts for both
# strict correctness and flexible wording. Higher score = more accurate answer.



# ---------- custom functions for various use cases ----------
def to_text(x) -> str:
    if x is None:
        return ""
    try:
        import pandas as _pd
        if _pd.isna(x):  
            return ""
    except Exception:
        pass
    return str(x)

def normalize_basic(s: str) -> str:
    s = to_text(s).lower().strip()
    s = re.sub(r"[^\w\s]", " ", s)      
    s = re.sub(r"\s+", " ", s)
    return s

def no_answer_template(s: str) -> bool:
    s = normalize_basic(s)
    pats = [
        "no answer","not in passage","not within passage","not stated",
        "cannot be determined","cannot be found","insufficient information",
        "unknown","not specified","no information","answer not available",
        "cannot determine","not provided","information is missing"
    ]
    return any(p in s for p in pats)


# Boolean mapping

BOOL_YES = {"yes","true","correct","right","affirmative","y"}
BOOL_NO  = {"no","false","incorrect","wrong","negative","n"}

def parse_boolean(s: str):
    t = normalize_basic(s)
    if t in BOOL_YES: return True
    if t in BOOL_NO:  return False
    return None


# Number & unit parsing

UNIT_FACTORS = {
    # length
    "mm": ("m", 1e-3), "cm": ("m", 1e-2), "m": ("m", 1.0), "km": ("m", 1e3),
    # mass
    "mg": ("kg", 1e-6), "g": ("kg", 1e-3), "kg": ("kg", 1.0),
    # volume
    "ml": ("l", 1e-3), "l": ("l", 1.0),
}
NUM_UNIT_RE = re.compile(r"(?P<num>\d+(?:\.\d+)?)\s*(?P<unit>[a-zA-Z]+)?")

def extract_numbers_units(s: str):
    """Return list of (value:float, base_unit:str|None). If unit known, convert to base unit."""
    out = []
    for m in NUM_UNIT_RE.finditer(to_text(s)):
        val = float(m.group("num"))
        u = m.group("unit").lower() if m.group("unit") else None
        if u in UNIT_FACTORS:
            base, factor = UNIT_FACTORS[u]
            out.append((val * factor, base))
        else:
            out.append((val, None))
    return out

def compare_single_number(gold_s: str, resp_s: str, rel_tol=0.02, abs_tol=0.5) -> float:
    g = extract_numbers_units(gold_s)
    r = extract_numbers_units(resp_s)
    if len(g) != 1 or len(r) != 1:
        return 0.0
    (gv, gu), (rv, ru) = g[0], r[0]
    if gu and ru and gu != ru:
        return 0.0
    diff = abs(rv - gv)
    if (abs(gv) >= 1 and diff/abs(gv) <= rel_tol) or diff <= abs_tol:
        return 1.0
    return 0.0

def compare_multi_numbers(gold_s: str, resp_s: str, rel_tol=0.02, abs_tol=0.5) -> float:
    G = extract_numbers_units(gold_s)
    R = extract_numbers_units(resp_s)
    if not G or not R:
        return 0.0
    hits = 0
    used = set()
    for rv, ru in R:
        best_j = None
        best_diff = float("inf")
        for j, (gv, gu) in enumerate(G):
            if j in used: 
                continue
            if (gu and ru) and (gu != ru):
                continue
            diff = abs(rv - gv)
            rel_ok = (abs(gv) >= 1 and diff/abs(gv) <= rel_tol)
            abs_ok = (diff <= abs_tol)
            if rel_ok or abs_ok:
                if diff < best_diff:
                    best_diff = diff
                    best_j = j
        if best_j is not None:
            hits += 1
            used.add(best_j)
    precision = hits / len(R)
    recall    = hits / len(G)
    if precision+recall == 0:
        return 0.0
    return 2*precision*recall/(precision+recall)


# Date parsing & comparison

def parse_date_ymd(s: str):
    if not dateparser:
        return None, None  
    raw = to_text(s).strip()
    if not raw: return None, None
    dt = dateparser.parse(raw)
    if not dt: return None, None
    # granularity: if only year present, or month-year, etc.
    gran = "day"
    if re.fullmatch(r"\d{4}", raw): gran = "year"
    elif re.search(r"^\w+\s+\d{4}$", raw, re.I) or re.search(r"^\d{4}-\d{2}$", raw):
        gran = "month"
    return dt, gran

def compare_dates(gold_s: str, resp_s: str) -> float:
    gdt, ggran = parse_date_ymd(gold_s)
    rdt, rgran = parse_date_ymd(resp_s)
    if not gdt or not rdt:
        return 0.0
    gran = ggran or "day"
    if gran == "year":
        return 1.0 if gdt.year == rdt.year else 0.0
    if gran == "month":
        return 1.0 if (gdt.year == rdt.year and gdt.month == rdt.month) else 0.0
    return 1.0 if (gdt.year == rdt.year and gdt.month == rdt.month and gdt.day == rdt.day) else 0.0


# Token F1 & ROUGE-L

def tokens(s: str):
    return normalize_basic(s).split()

def token_f1(gold: str, resp: str) -> float:
    g = tokens(gold); r = tokens(resp)
    if not g and not r: return 1.0
    if not g or not r:  return 0.0
    g_counts = {}
    for t in g: g_counts[t] = g_counts.get(t, 0) + 1
    match = 0
    for t in r:
        if g_counts.get(t, 0) > 0:
            match += 1
            g_counts[t] -= 1
    precision = match / len(r)
    recall    = match / len(g)
    if precision+recall == 0:
        return 0.0
    return 2*precision*recall/(precision+recall)

def rouge_l_f1(gold: str, resp: str) -> float:
    # LCS length
    g, r = tokens(gold), tokens(resp)
    if not g or not r: return 0.0
    m, n = len(g), len(r)
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(m):
        for j in range(n):
            if g[i] == r[j]:
                dp[i+1][j+1] = dp[i][j] + 1
            else:
                dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
    lcs = dp[m][n]
    prec = lcs / n
    rec  = lcs / m
    if prec+rec == 0: return 0.0
    return 2*prec*rec/(prec+rec)


# List / set handling

ITEM_SEP_RE = re.compile(r"[;\n]|(?:\s*,\s*)|(?:\s+and\s+)|(?:•)")

def split_items(s: str):
    # split by commas/semicolons/newlines/bullets/ 'and'
    parts = [p.strip() for p in ITEM_SEP_RE.split(to_text(s)) if p and p.strip()]
    return parts

def list_f1(gold_s: str, resp_s: str) -> float:
    g_items = [normalize_basic(x) for x in split_items(gold_s)]
    r_items = [normalize_basic(x) for x in split_items(resp_s)]
    if not g_items and not r_items: return 1.0
    if not g_items or not r_items:  return 0.0
    # greedy exact-or-token-overlap matching
    hits = 0
    used = set()
    for r in r_items:
        best_j = None
        best_sim = 0.0
        r_tok = set(tokens(r))
        for j, g in enumerate(g_items):
            if j in used: continue
            g_tok = set(tokens(g))
            inter = len(r_tok & g_tok)
            union = len(r_tok | g_tok) or 1
            sim = inter/union
            if r == g: sim = 1.0
            if sim > best_sim:
                best_sim = sim
                best_j = j
        if best_j is not None and best_sim >= 0.6:
            hits += 1
            used.add(best_j)
    precision = hits / len(r_items)
    recall    = hits / len(g_items)
    if precision+recall == 0: return 0.0
    return 2*precision*recall/(precision+recall)


# Type detection

def detect_type(gold_raw: str):
    g = to_text(gold_raw).strip()
    g_norm = normalize_basic(g)
    # multi-gold aliases
    if "||" in g:
        return "aliases"
    # boolean?
    if parse_boolean(g_norm) is not None:
        return "boolean"
    # date?
    if dateparser and dateparser.parse(g):
        return "date"
    # numbers?
    nums = extract_numbers_units(g)
    if len(nums) >= 2:
        return "numeric_multi"
    if len(nums) == 1:
        return "numeric_single"
    # list?
    if len(split_items(g)) >= 2:
        return "list"
    # short/long text
    tok_len = len(tokens(g))
    return "short_text" if tok_len <= 5 else "long_text"


# Core accuracy scoring for a single (gold, resp) pair

def score_pair(gold_raw: str, resp_raw: str):
    g = to_text(gold_raw); r = to_text(resp_raw)
    g_norm = normalize_basic(g); r_norm = normalize_basic(r)

    t = detect_type(g)
    diag = {"acc_type_used": t}

    if t == "aliases":
        # split by '||', take best score across alternatives
        cands = [x.strip() for x in g.split("||") if x.strip()]
        best = 0.0; best_t = None
        for cand in cands:
            s, subdiag = score_pair(cand, r)
            if s > best:
                best = s; best_t = subdiag.get("acc_type_used", "unknown")
        diag["acc_type_used"] = f"aliases->{best_t}"
        return best, diag

    if t == "boolean":
        gb = parse_boolean(g_norm); rb = parse_boolean(r_norm)
        score = 1.0 if (rb is not None and rb == gb) else 0.0
        return score, diag

    if t == "date" and dateparser:
        score = compare_dates(g, r)
        return score, diag

    if t == "numeric_single":
        score = compare_single_number(g, r)
        return score, diag

    if t == "numeric_multi":
        score = compare_multi_numbers(g, r)
        return score, diag

    if t == "list":
        score = list_f1(g, r)
        return score, diag

    if t == "short_text":
        em = 1.0 if g_norm == r_norm else 0.0
        f1 = token_f1(g, r)
        score = max(em, (0.5*em + 0.5*f1))
        return score, {**diag, "em": em, "token_f1": f1}

    # long_text
    f1 = token_f1(g, r)
    rl = rouge_l_f1(g, r)
    score = 0.6*f1 + 0.4*rl
    return score, {**diag, "token_f1": f1, "rouge_l": rl}


# Row-level accuracy score (handles unanswerable & no_gold)

def accuracy_score_row(row: pd.Series, tau_label=0.8):
    gold_raw = to_text(row.get("gold_reference",""))
    resp_raw = to_text(row.get("ai_response",""))
    no_gold  = str(to_text(row.get("no_gold",""))).strip().lower() in {"1","true","yes","y","t"}

    # if no gold → treat accuracy as 0 (not applicable but penalized)
    if no_gold or gold_raw.strip() == "":
        return {
            "acc_score": 0.0,
            "acc_label": 0,
            "acc_type_used": "n/a",
            "unanswerable": False
        }

    # if gold indicates unanswerable
    g_unans = no_answer_template(gold_raw)
    r_unans = no_answer_template(resp_raw)
    if g_unans:
        score = 1.0 if r_unans else 0.0
        return {
            "acc_score": score,
            "acc_label": int(score >= tau_label),
            "acc_type_used": "unanswerable",
            "unanswerable": True
        }

    score, diag = score_pair(gold_raw, resp_raw)
    score = float(min(1.0, max(0.0, score)))
    label = int(score >= tau_label)
    return {"acc_score": score, "acc_label": label, **diag, "unanswerable": False}


def add_accuracy_scores(df: pd.DataFrame, tau_label=0.8) -> pd.DataFrame:
    rows = [accuracy_score_row(r, tau_label=tau_label) for _, r in df.iterrows()]
    return pd.concat([df.reset_index(drop=True), pd.DataFrame(rows)], axis=1)

In [5]:
df = add_accuracy_scores(df, tau_label=0.8)
df[["gold_reference","ai_response","acc_score","acc_label","acc_type_used"]].head()

gold_reference                                        ai_response  \
0            NaN                    The passage doesn’t state this.   
1            NaN  It states that what did the court of justice w...   
2            NaN  The decision was influenced by the Paris Conve...   
3            NaN                               No answer in passage   
4            NaN  Bullet 1) The Court accepted horizontal effect...   

   acc_score  acc_label acc_type_used  
0        0.0          0           n/a  
1        0.0          0           n/a  
2        0.0          0           n/a  
3        0.0          0           n/a  
4        0.0          0           n/a

In [6]:
df.to_csv("scored datasets/accuracy_scores_added.csv")